<a href="https://colab.research.google.com/github/Gobihanath/DTect/blob/main/Dtect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

input_shape = (512, 512, 3)    # 3 means the rgb format



## Importing Dataset from google drive

In [ ]:
from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


In [75]:
Image_Width = 512
Image_Height = 512
Image_Size = (Image_Width, Image_Height)
Image_Channel = 3
batch_size=20

# **Image Data Preprocessing**

In [76]:
data_set = tf.keras.preprocessing.image_dataset_from_directory(
    "/content/drive/MyDrive/Dataset",
    seed=123,         #If you want to ensure that the shuffling of your dataset is the same across different runs, you can keep the seed value constant. If you don't care about reproducibility, you can omit the seed parameter or use a different value for each run.
    shuffle=True,
    image_size=Image_Size,
    batch_size=batch_size
)


Found 1200 files belonging to 6 classes.


In [79]:
train_size = int(len(data_set)*.8)
val_size = int(len(data_set)*.1)
test_size = int(len(data_set)*.1)

In [80]:
train_size



48

## Dataset Partition

In [77]:
def get_dataset_partitions_tf(data_set, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(data_set)

    if shuffle:
        data_set = data_set.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = data_set.take(train_size)
    val_ds = data_set.skip(train_size).take(val_size)
    test_ds = data_set.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

In [78]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(data_set)

print(len(train_ds))
print(len(val_ds))
print(len(test_ds))

48
6
6


## Training, Validation,Testing Data Preprocessing

In [81]:
train_datagen = ImageDataGenerator(rotation_range=15,
                                  rescale=1./255,
                                  shear_range=0.1,
                                  zoom_range=0.2,
                                  horizontal_flip=True,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,)

train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Dataset',
                                              target_size=Image_Size,
                                              batch_size=20,
                                              class_mode = 'categorical')


test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('/content/drive/My Drive/Dataset',
                                                  target_size=Image_Size,
                                                  batch_size = 20,
                                                  class_mode='categorical')

Found 1200 images belonging to 6 classes.
Found 1200 images belonging to 6 classes.


# Building The CNN model

In [ ]:
model = Sequential()

# add convolutional layers with pooling and dropout
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# flatten the output and add dense layers with dropout
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

#This is the output layer
model.add(Dense(units=10, activation='softmax'))      #units=10 means we have 10 classes(signatures) softmax is used for

# compile the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# print the model summary
model.summary()

# **Training and Validation**